In [1]:
import pandas as pd

In [2]:
#orders_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/orders.csv'
#products_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/products.csv'
#customers_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/customers.csv'
#regions_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/regions.csv'

#orders = pd.read_csv(orders_url)
#products = pd.read_csv(products_url)
#customers = pd.read_csv(customers_url)
#regions = pd.read_csv(regions_url)

# directly read csv seems to be faster
orders = pd.read_csv('../DEAD/data/orders.csv')
products = pd.read_csv('../DEAD/data/products.csv')
customers = pd.read_csv('../DEAD/data/customers.csv')
regions = pd.read_csv('../DEAD/data/regions.csv')

In [3]:
#products.Type.unique()

In [4]:
#customers['Key Account'].unique()

In [5]:
# Splitting Orders Table by Products
orders[['Products_split']] = orders[['Products']].apply(lambda x: x.str.split(';'))
orders[['productsIDs_split', 'Quantities_splits', 'ProductPricesInCP_split']] = orders[['productsIDs', 'Quantities', 'ProductPricesInCP']].apply(lambda x: x.str.split(','))

orders = orders.explode(['Products_split','productsIDs_split', 'Quantities_splits', 'ProductPricesInCP_split'])
orders.drop(['Products','productsIDs', 'Quantities', 'ProductPricesInCP'],axis=1,inplace=True)

orders[['productsIDs_split', 'Quantities_splits']] = orders[['productsIDs_split', 'Quantities_splits']] .astype(int)
orders['ProductPricesInCP_split'] = orders['ProductPricesInCP_split'] .astype(float)
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
orders['DeliveryDate'] = pd.to_datetime(orders['DeliveryDate'])

In [6]:
# Join all other tables

combine = orders.merge(products, left_on ='productsIDs_split', right_on = 'product_code', how = 'left')\
                .merge(customers.drop(['Territory'],axis=1), left_on ='CustomerID', right_on = 'Account Code', how = 'left')\
                .merge(regions, on = 'Territory', how = 'left')

In [7]:
combine.shape

(878427, 30)

In [8]:
pd.set_option('display.max_columns', None)
combine.head()

,OrderID,OrderDate,DeliveryDate,CustomerID,Territory,CartPriceInCP,CartPrice,Products_split,productsIDs_split,Quantities_splits,ProductPricesInCP_split,product_code,Business Unit,Type,Products Business Line Leader,Subtype,Product Name,Brand Name,Brand Manager,Account Code,Account Type,Key Account,Key Account Manager,Account Name,Account Manager,Nation,Region,Regional Manager,Area,Area Manager
0,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,Blood of the Lycanthrope Antidote,12436254,8,1128.0,12436254,ADVENTURING,POTIONS & SCROLLS,Bruno Notlot,POTIONS & ALCHEMY,Blood of the Lycanthrope Antidote,MYTHAL,Arentian Mythrandius,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
1,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,Blanket,24180928,4,2728.0,24180928,ADVENTURING,ADVENTURING EQUIPMENT,Bubbles,SURVIVAL & WILDERNESS,Blanket,U.N.N.,Ruralion Baeneduren,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
2,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,Robe of Serpents,97076903,9,5517.0,97076903,ADVENTURING,ADVENTURING EQUIPMENT,Bubbles,CLOAK,Robe of Serpents,U.N.N.,Ruralion Baeneduren,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
3,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,"Mirror, steel",33879927,3,912.0,33879927,ADVENTURING,TOOLS & KITS,Kay Glimshine,TOOLS,"Mirror, steel",RED CURTAIN,Siras Invictus,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
4,2,2019-02-03,2019-02-05,30949079,Arrabar,1196,11 GP; 9 SP; 6 CP,Sword of Life Stealing,43737386,1,91.0,43737386,LUXURY SPECIALTIES,ARMS & ARMOUR,T'avin T'soban,MARTIAL MELEE WEAPON,Sword of Life Stealing,ARCANTIC,Enthoril Murkywaters,30949079,Key Account,Clothes Contact,Asuma Bahamut,Clothes Contact Arrabar,NaN,Chondath,Calim Empire,Mitchell Brown,South,Cipriano Seidel


In [9]:
combine.to_csv('orders_merge.csv',index=False)
combine.to_json('orders_merge.json', orient='records',index=False)

In [10]:
# special preprocessing for graph AM_5

# create year
combine['order_year']=combine['OrderDate'].dt.year

In [11]:
# groupby year
cols = ['order_year', 'Type', 'Key Account','ProductPricesInCP_split']
data_viz1 = combine[cols]
data_viz1 = data_viz1.groupby(['order_year','Type', 'Key Account']).sum().reset_index()

In [12]:
data_viz1 = data_viz1.sort_values(by='order_year')
data_viz1.head()

,order_year,Type,Key Account,ProductPricesInCP_split
0,2019,ADVENTURING EQUIPMENT,Bardic College Union,3086829.0
81,2019,POTIONS & SCROLLS,Sprite's & Spirits,1143734.0
80,2019,POTIONS & SCROLLS,Private Buyer,5711577.0
79,2019,POTIONS & SCROLLS,Order of the Gauntlet,870255.0
78,2019,POTIONS & SCROLLS,Nobility,1398002.0


In [13]:
# Calculate year-on-year growth
# grouped = data_viz1.groupby(['Type', 'Key Account'])
# data_viz1['yoy_growth'] = grouped['ProductPricesInCP_split'].pct_change()
# data_viz1.head()

In [14]:
# Calculate 2023-2019 growth
data_19 = data_viz1[data_viz1['order_year'] == 2019]
data_23 = data_viz1[data_viz1['order_year'] == 2023]
data_viz1 = data_23.drop('order_year',axis=1).merge(data_19.drop('order_year',axis=1), on = ['Type','Key Account'],suffixes=('_23', '_19'))
data_viz1['growth_tot'] = data_viz1['ProductPricesInCP_split_23']/data_viz1['ProductPricesInCP_split_19'] - 1

In [15]:
data_viz1.head()

,Type,Key Account,ProductPricesInCP_split_23,ProductPricesInCP_split_19,growth_tot
0,POTIONS & SCROLLS,Black Phoenix,1269059.0,1218962.0,0.041098
1,POTIONS & SCROLLS,Bottle's Up,1247017.0,1205211.0,0.034688
2,POTIONS & SCROLLS,Clothes Contact,1455144.0,1471400.0,-0.011048
3,POTIONS & SCROLLS,House of Wonders,1588633.0,1892445.0,-0.160539
4,POTIONS & SCROLLS,Lionshield Coster,1262351.0,1165737.0,0.082878


In [16]:
data_viz1.to_csv('data_viz1.csv',index=False)
data_viz1.to_json('data_viz1.json', orient='records',index=False)